## Tom's work starts here.

### Gather dependencies

In [ ]:
# Dependencies
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
# from sodapy import Socrata
from config import census_key
import pprint
import requests
import csv
import os

### Function for quickly calling a DataFrame

In [ ]:
# Function for quickly assempling a DataFrame
def json_to_dataframe(response):
    return pd.DataFrame(response.json()[1:], columns=response.json()[0])

### Create DataFrame for CDC Data

In [ ]:
cdc_data = pd.read_csv("../Project_1/cdc_data.csv")
cdc_data
cdc_df = pd.DataFrame(cdc_data)
cdc_df

# Choose the columns we want and rename anything that isn't clear
cdc_df = cdc_df[['case_month', 'res_state', 'res_county', 'age_group', 'sex', 'race', 'ethnicity', 'current_status']]
cdc_df = cdc_df.rename(columns={
    'res_state': 'State',
    'res_county': 'County',
    'ethnicity': 'Ethnicity',
    'current_status': 'Infection Status',
    'race':'Race'
})

# Drop "Unkown" race from dataset
cdc_df = cdc_df.loc[cdc_df['Race']!= 'Unknown', :]

# Set Infections to 1 and replace the various races with values that are compatible with the census data
cdc_df['Infections'] = 1
cdc_df['Race'] = cdc_df['Race'].replace('White', 'Caucasian')
cdc_df['Race'] = cdc_df['Race'].replace('Black', 'African American')
cdc_df['Race'] = cdc_df['Race'].replace('American Indian/Alaska Native', 'Native American')
cdc_df['Race'] = cdc_df['Race'].replace('Multiple/Other', 'Other')
cdc_df['Race'] = np.where(np.logical_and(cdc_df['Race'] == 'Caucasian', cdc_df['Ethnicity'] == 'Hispanic/Latino'), 'Hispanic', cdc_df['Race'])

# NOTE: It's not clear what happens with Hawaiian here. 

cdc_df.head()

### Drop NaN and compile totals by county and race

In [ ]:
# Drop NaNs and then group by County and Race
cdc_df = cdc_df.dropna(how='any')
cdc_df = cdc_df[['County', 'Race', 'Infections']]
cdc_grouped_county = cdc_df.groupby(['County', 'Race'])
cdc_df = cdc_grouped_county.count()
cdc_df.reset_index(inplace=True)
cdc_df.head()

### Create poverty DataFrame based on US Census

In [ ]:
# Poverty DataFrame By Race - 2020

url = "https://api.census.gov/data/2020/acs/acs5?get=NAME,B17001_002E,B17001A_002E,B17001B_002E,B17001C_002E,B17001D_002E,B17001E_002E,B17001F_002E,B17001G_002E,B17001H_002E,B17001I_002E&for=county:*&in=state:27&key={0}".format(census_key)
response = requests.request("GET", url)
response
poverty_df = json_to_dataframe(response)

# Split NAME into county & state
name = poverty_df['NAME'].str.split(",", n=1, expand=True)
COUNTY = poverty_df['County'] = name[0]
STATE = poverty_df['State'] = name[1]
poverty_df.drop(columns=["NAME"], inplace=True)
poverty_df['Caucasian2'] = poverty_df['B17001A_002E'].astype(int) - poverty_df['B17001I_002E'].astype(int)
poverty_df['Other'] = poverty_df['B17001F_002E'].astype(int) + poverty_df['B17001G_002E'].astype(int)

# Get the columns we want
poverty_df = poverty_df[['County', 'State','B17001_002E','Caucasian2', 'B17001B_002E','B17001C_002E','B17001D_002E','B17001E_002E','Other','B17001I_002E']]

# Rename columns to something intelligible
poverty_df = poverty_df.rename(columns={
    'B17001_002E':'Poverty Total',
    'Caucasian2':'Caucasian',
    'B17001B_002E':'African American',
    'B17001C_002E':'Native American',
    'B17001D_002E':'Asian',
    'B17001E_002E':'Hawaiian', # I had to change this from Native Hawaiian to Hawaiian for compatibility with the rest - Dom
    'B17001I_002E':'Hispanic' 
    })

# # Remove ...County from County in dataframe
poverty_df['County'] = poverty_df['County'].replace('County', '', regex=True)
# NOTE: The line above accidentally leaves a blank space at the end of these county names! That space caused a merge to fail
# on any of these counties because, for example, "AITKIN " is not the same as "AITKIN". 

# Remove any blank space that may be present at the begining or end of the county name
poverty_df['County'] = poverty_df['County'].str.strip()

# And then convert the county name to uppercase
poverty_df['County'] = poverty_df['County'].str.upper()

poverty_df['Year'] = 2020
#poverty_df.to_csv("../Project_1/population_in_poverty.csv")
poverty_df.head()

In [ ]:
# Choose only the columns we need
poverty_df = poverty_df[['County', 'Caucasian', 'African American', 'Native American', 'Asian', 'Hawaiian', 'Other', 'Hispanic']]
poverty_df.head()

In [ ]:
# Use melt to convert this into a format similar to the CDC data
poverty_df = poverty_df.melt(id_vars='County', value_vars=['Caucasian', 'African American', 'Native American', 'Asian', \
                                                    'Hawaiian', 'Other', 'Hispanic'], \
                                                     var_name='Race', value_name='Poverty')
poverty_gb = poverty_df.groupby(['County', 'Race'])
poverty_df = poverty_gb.sum()
poverty_df.reset_index(inplace=True)
poverty_df.head()

### Create population DataFrame by county and race

In [ ]:
# Population by county, by race
url = "https://api.census.gov/data/2020/acs/acs5?get=NAME,B03002_001E,B03002_003E,B03002_004E,B03002_005E,B03002_006E,B03002_007E,B03002_008E,B03002_009E,B03002_012E&for=county:*&in=state:27&key={0}".format(census_key)


response = requests.request("GET", url)
response
response_df = json_to_dataframe(response)
response_df = response_df.rename(columns={
    'B03002_001E': 'Population Total',
    'B03002_003E': 'Caucasian',
    'B03002_004E': 'African American',
    'B03002_005E': 'Native American',
    'B03002_006E': 'Asian',
    'B03002_007E': 'Hawaiian',
    'B03002_012E': 'Hispanic'
})
response_df['Other'] = response_df['B03002_008E'].astype(int) + response_df['B03002_009E'].astype(int)
mn_pop_df = response_df

# Split NAME into county & state
name = mn_pop_df['NAME'].str.split(",", n=1, expand=True)
COUNTY = mn_pop_df['County']=name[0]
STATE = mn_pop_df['State']=name[1]
mn_pop_df.drop(columns=["NAME"], inplace=True)

# # Remove ...County from County in dataframe
mn_pop_df['County'] = mn_pop_df['County'].replace('County', '', regex=True)
# NOTE: The line above accidentally leaves a blank space at the end of these county names! That space caused a merge to fail
# on any of these counties because, for example, "AITKIN " is not the same as "AITKIN". 

# Remove any blank space that may be present at the begining or end of the county name
mn_pop_df['County'] = mn_pop_df['County'].str.strip()

# # Upper case for County
mn_pop_df['County'] = mn_pop_df['County'].str.upper()
mn_pop_df = mn_pop_df[['County', 'Population Total', 'Caucasian', 'African American', 'Native American', 'Asian', 'Hawaiian', 'Hispanic', 'Other']]
mn_pop_df.to_csv("../Project_1/mn_population by race.csv")
mn_pop_df.head()


### Coordinates for heat map -- if time permits

In [ ]:
# # Get the coordinates of counties in MN
# url = "https://en.wikipedia.org/wiki/User:Michael_J/County_table"

# table = pd.read_html(url)
# df = table[0]
# Counties = df.loc[df['State'] == "MN", : ]
# counties_df = Counties[['County [2]', 'Latitude', 'Longitude']]
# counties_df#.to_csv("../Project_1/county_geo.csv")  

In [ ]:
# Choose only the columns we need
mn_pop_df = mn_pop_df[['County', 'Caucasian', 'African American', 'Native American', 'Asian', 'Hawaiian', 'Other', 'Hispanic']]
mn_pop_df.head()

In [ ]:
# Use melt to convert this into a format similar to the CDC data
pop_df = mn_pop_df.melt(id_vars='County', value_vars=['Caucasian', 'African American', 'Native American', 'Asian', \
                                                      'Hawaiian', 'Other', 'Hispanic'], \
                                                      var_name='Race', value_name='Population')
pop_gb = pop_df.groupby(['County', 'Race'])
pop_df = pop_gb.sum()
pop_df.reset_index(inplace=True)
pop_df.head()

### Merge the datasets

In [ ]:
merged_df = pd.merge(pop_df, poverty_df, on=['County', 'Race'])
merged_df = pd.merge(merged_df, cdc_df, on=['County', 'Race'])
merged_df.head()

In [ ]:
# Edited merge_df for nan infections to be 0
merged_df = pd.merge(pop_df, poverty_df, on=['County', 'Race'])
merged_df = pd.merge(merged_df, cdc_df, on=['County', 'Race'], how = "left")
merged_df["Infections"] = merged_df["Infections"].fillna(0)
merged_df

# Convert all numerical values in merged_df from strings to int
merged_df["Population"] = merged_df["Population"].astype(int)
merged_df["Poverty"] = merged_df["Poverty"].astype(int)
merged_df["Infections"] = merged_df["Infections"].astype(int)

# Calculate poverty and infection rates
poverty_rate = []
infection_rate = []

i = 0
while i < len(merged_df):
    if merged_df["Population"][i] == 0:
        poverty_rate.append(0)
        infection_rate.append(0)
    else: 
        poverty_rate.append(merged_df["Poverty"][i] / merged_df["Population"][i])
        infection_rate.append(merged_df["Infections"][i] / merged_df["Population"][i])
    i += 1
    
merged_df["Poverty Rate"] = poverty_rate
merged_df["Infection Rate"] = infection_rate

# merged_df[merged_df["County"] == "CROW WING"]    
merged_df.head()

In [ ]:
# Convert all non-caucasian races to "non-caucasian"
#non_caucasian = ['African American', 'Hispanic', 'Asian', 'Native American', 'Hawaiian', 'Other']
merged_df['Two-Races']=["Non-Caucasian" if (x=="African American" or x=="Asian" or x=="Hispanic" or x=="Native American"\
    or x=="Hawaiian" or x=="Other") else "Caucasian" for x in merged_df['Race']]
merged_df.head()

In [ ]:
# Metro area counties
merged_df['Metro-Non']=["Metro" if (x=="RAMSEY" or x=="DAKOTA" or x=="HENNEPIN" or x=="ANOKA"\
    or x=="SCOTT" or x=="WASHINGTON" or x=="CARVER") else "Non_Metro" for x in merged_df['County']]
merged_df_metro = merged_df[['Metro-Non', 'County', 'Two-Races', 'Population', 'Poverty', 'Infections']]
merged_df_metro.head()

In [ ]:
race_df = merged_df_metro[['Two-Races', 'Population']].groupby(['Two-Races'])
race_df.sum().reset_index()

metro_df = merged_df_metro[['Metro-Non', 'Population']].groupby(['Metro-Non'])
metro_df.sum().reset_index()

In [ ]:
two_races_df = merged_df[['County', 'Two-Races', 'Population', 'Poverty', 'Infections']]
two_races_df.head()

In [ ]:
# pd.set_option('mode.chained_assignment', None)
# poverty_rate = []
# infection_rate = []

# i = 0
# while i < len(two_races_df):
#     if two_races_df["Population"][i] == 0:
#         poverty_rate.append(0)
#         infection_rate.append(0)
#     else: 
#         poverty_rate.append(two_races_df["Poverty"][i] / two_races_df["Population"][i])
#         infection_rate.append(two_races_df["Infections"][i] / two_races_df["Population"][i])
#     i += 1
    
# two_races_df["Poverty Rate"] = poverty_rate
# two_races_df["Infection Rate"] = infection_rate

In [ ]:
# caucasian = two_races_df.loc[two_races_df['Two-Races'] == 'Caucasian']

# # Population, poverty, and infection totals
# row_sum = two_races_df.iloc[:,2:5].sum()

# two_races_df.loc['Total'] = row_sum
# two_races_df.fillna('')
# two_races_df.loc['Total', 'County']=''
# two_races_df.loc['Total', 'Two-Races'] = ''
# two_races_df.loc['Total', 'Poverty Rate'] = ''
# two_races_df.loc['Total', 'Infection Rate'] = ''
# two_races_df

# Mauvonte's Code Starts Here

In [ ]:
#Mean and Median
mean_df = merged_df.groupby('Race').mean()
mean_df.reset_index()

In [ ]:
#find quartiles and upper, lower bound, outliers

def boxplot(race):
    covid_box = merged_df.loc[merged_df['Race'] == race]['Infections']
    quartiles = covid_box.quantile([.25,.5,.75])
    lowerq = quartiles[0.25]
    upperq = quartiles[0.75]
    iqr = upperq - lowerq
    lower_bound = lowerq - (1.5 * iqr)
    upper_bound = upperq + (1.5 * iqr)
    outliers = covid_box.loc[(covid_box < lower_bound) | (covid_box > upper_bound)]
    print(outliers)
    return covid_box


race_1 = boxplot('African American')
race_2 = boxplot('Caucasian')
race_3 = boxplot('Asian')
race_4 = boxplot('Hispanic')
race_5 = boxplot('American Indian/Alaskan Native')
race_6 = boxplot('Hawaiian')
race_7 = boxplot('Other')

plt.boxplot([race_1, race_2, race_3, race_4, race_5, race_6, race_7], labels=['Black', 'White', 'Asian', 'Hispanic', 'American Indian/Alaskan Native', 'Hawaiian', 'Other'])
plt.ylabel('Infection by Counties')
plt.title('Covid Results Based on Ethnicity in Minnensota')
plt.xticks(rotation=90)
plt.show()


# Putting stat summary into dataframe


In [ ]:
# Statistics by county
#lief's groupby
merged_county_gb = merged_df.groupby(["County"])

county_pop_stats = merged_county_gb["Population"].agg(["max", "min", "mean", np.median, "var", "std", "sem"])
county_pov_stats = merged_county_gb["Poverty"].agg(["max", "min", "mean", np.median, "var", "std", "sem"])
county_inf_stats = merged_county_gb["Infections"].agg(["max", "min", "mean", np.median, "var", "std", "sem"])
county_pov_rate_stats = merged_county_gb["Poverty Rate"].agg(["max", "min", "mean", np.median, "var", "std", "sem"])
county_inf_rate_stats = merged_county_gb["Infection Rate"].agg(["max", "min", "mean", np.median, "var", "std", "sem"])

In [ ]:
# county_inf_rate_stats.T

In [ ]:
# merge the county_pov_stats df with county_pove_rate_stats
merged_county_pov_stats_df = pd.merge(county_pov_stats, county_pov_rate_stats, on='County',  how='inner')

#rename columns
col_names = {'max_x':'Cty_Pov_Stats_Max',
            'min_x' :'Cty_Pov_Stats_Min',
            'mean_x' :'Cty_Pov_Stats_Mean',
            'median_x':'Cty_Pov_Stats_Median',
            'var_x':'Cty_Pov_Stats_Variance',
            'std_x':'Cty_Pov_Stats_Std',
            'sem_x':'Cty_Pov_Stats_SEM',
            'max_y':'Cty_Pov_Rate_Max',
            'min_y':'Cty_Pov_Rate_Min',
            'mean_y':'Cty_Pov_Rate_Mean',
            'median_y':'Cty_Pov_Rate_Median',
            'var_y':'Cty_Pov_Rate_Variance',
            'std_y':'Cty_Pov_Rate_Std',
            'sem_y':'Cty_Pov_Rate_SEM'}

merged_county_pov_stats_df = merged_county_pov_stats_df.rename(columns=col_names)
#merged_county_pov_stats_df

In [ ]:
#merge county_inf_stats df with county_inf_rate_stats 

merged_county_inf_stats_df = pd.merge(county_inf_stats, county_inf_rate_stats, on='County', how='inner')

#rename columns

col_names2 = {
            'max_x':'Cty_Infect_Stats_Max',
            'min_x':'Cty_Infect_Stats_Min',
            'mean_x':'Cty_Infect_Stats_Mean',
            'median_x':'Cty_Infect_Stats_Median',
            'var_x':'Cty_Infect_Stats_Variance',
            'std_x':'Cty_Infect_Stats_Std',
            'sem_x':'Cty_Infect_Stats_SEM',
            'max_y':'Cty_Infect_Rate_Max',
            'min_y':'Cty_Infect_Rate_Min',
            'mean_y':'Cty_Infect_Rate_Mean',
            'median_y':'Cty_Infect_Rate_Median',
            'var_y':'Cty_Infect_Rate_Variance',
            'std_y':'Cty_Infect_Rate_Std',
            'sem_y':'Cty_Infect_Rate_SEM'}

merged_county_inf_stats_df = merged_county_inf_stats_df.rename(columns=col_names2)
#merged_county_inf_stats_df

In [ ]:
#merge both df containing the county poverty and county infection
merged_pov_inf_df = pd.merge(merged_county_inf_stats_df, merged_county_pov_stats_df, on='County', how='left')
#merged_pov_inf_df

In [ ]:
#merge the merged_pov_inf_df with county_pop_stats
merged_county_data = pd.merge(merged_pov_inf_df, county_pop_stats, on='County', how='left')
#rename population columns
col_names3 = {
            'max':'Cty_Population_Max',
            'min':'Cty_Population_Min',
            'mean':'Cty_Population_Mean',
            'median':'Cty_Population_Median',
            'var': 'Cty_Population_Variance',
            'std': 'Cty_Population_Std',
            'sem': 'Cty_Population_SEM',
            }
merged_county_data = merged_county_data.rename(columns=col_names3)
merged_county_data

In [ ]:
# Statistics by race

merged_race_gb = merged_df.groupby(["Race"])

race_pop_stats = merged_race_gb["Population"].agg(["max", "min", "mean", np.median, "var", "std", "sem"])
race_pov_stats = merged_race_gb["Poverty"].agg(["max", "min", "mean", np.median, "var", "std", "sem"])
race_inf_stats = merged_race_gb["Infections"].agg(["max", "min", "mean", np.median, "var", "std", "sem"])
race_pov_rate_stats = merged_race_gb["Poverty Rate"].agg(["max", "min", "mean", np.median, "var", "std", "sem"])
race_inf_rate_stats = merged_race_gb["Infection Rate"].agg(["max", "min", "mean", np.median, "var", "std", "sem"])

In [ ]:
#merge race poverty df's 
merged_race_pov_stats = pd.merge(race_pov_stats, race_pov_rate_stats, on='Race', how='inner')
rc_names = {
            'max_x':'Race_Poverty_Max',
            'min_x':'Race_Poverty_Min',
            'mean_x':'Race_Poverty_Mean',
            'median_x':'Race_Poverty_Median',
            'var_x': 'Race_Poverty_Variance',
            'std_x': 'Race_Poverty_Std',
            'sem_x': 'Race_Poverty_SEM',
            'max_y': 'Race_Poverty_Rate_Max',
            'min_y': 'Race_Poverty_Rate_Min',
            'mean_y': 'Race_Poverty_Rate_Mean',
            'median_y': 'Race_Poverty_Rate_Median',
            'var_y': 'Race_Poverty_Rate_Variance',
            'std_y': 'Race_Poverty_Rate_Std',
            'sem_y': 'Race_Poverty_Rate_SEM'
            }
merged_race_pov_stats = merged_race_pov_stats.rename(columns=rc_names)
merged_race_pov_stats

In [ ]:
#merge infection df's
merged_race_inf_stats = pd.merge(race_inf_stats,race_inf_rate_stats, on='Race', how='inner')
#rename columns
rc_names2 = {
            'max_x':'Race_Infect_Max',
            'min_x':'Race_Infect_Min',
            'mean_x':'Race_Infect_Mean',
            'median_x':'Race_Infect_Median',
            'var_x':'Race_Infect_Variance',
            'std_x':'Race_Infect_Std',
            'sem_x':'Race_Infect_SEM',
            'max_y':'Race_Infect_Rate_Max',
            'min_y':'Race_Infect_Rate_Min',
            'mean_y':'Race_Infect_Rate_Mean',
            'median_y':'Race_Infect_Rate_Median',
            'var_y':'Race_Infect_Rate_Variance',
            'std_y':'Race_Infect_Rate_Std',
            'sem_y':'Race_Infect_Rate_SEM'
            }

merged_race_inf_stats = merged_race_inf_stats.rename(columns=rc_names2)
merged_race_inf_stats

In [ ]:
#merge race infect df's with race poverty df's
merged_race_pov_inf = pd.merge(merged_race_inf_stats, merged_race_pov_stats, on='Race', how='left')
merged_race_pov_inf

In [ ]:
#merge previous df with race pop df
merged_race_data = pd.merge(merge_race_pov_inf, race_pop_stats, on='Race', how='left')
#rename population columns
rc_names3 = {
            'max':'Race_Pop_Max',
            'min':'Race_Pop_Min',
            'mean':'Race_Pop_Mean',
            'median':'Race_Pop_Median',
            'var':'Race_Pop_Variance',
            'std':'Race_Pop_Std',
            'sem':'Race_Pop_SEM'
            }
merged_race_data = merged_race_pov_inf_pop.rename(columns=rc_names3)
merged_race_data


In [ ]:
#merge both race data and county data

county_race_df = pd.merge(merged_county_data, merged_race_pov_inf_pop, left_on=['County'] , right_on=['Race'], how='outer')
county_race_df.T

## Mai's Code starts Here

### Bar Charts on Race and Poverty

In [ ]:
# Generate a bar plot showing the total population of race groups, poverty and analyzing the differences of each groups for the merged dataframe
bar_plot_data = merged_df.groupby(['Race']).sum()["Poverty"]
plt.title("The Number of People in Poverty in Minnesota")
plt.xlabel("Race")
plt.ylabel("Poverty")
bar_plot_data.plot.bar(figsize=(8,5),color=('red'), rot=45)

In [ ]:
plot_df = merged_df.groupby(['Race']).sum()
plot_df

In [ ]:
max_pop = plot_df['Population'].max()
max_pop

In [ ]:
# Generate a bar chart to show the max population, poverty and infections of each race in MN
plot_df[['Population','Poverty','Infections']].plot.bar(figsize=(10,15))
plt.xlabel("Race")
plt.ylabel("Population (in millions)")

In [ ]:
# Generate a 3 bar chart to show poverty rate and infection rate for each of the race in MN
plot_df[['Poverty Rate','Infection Rate']].plot.bar(figsize=(10,15))
plt.xlabel("Race")
plt.ylabel("Rate")

## Mai's Boxplots
### Boxplots Based on Race and Poverty

In [ ]:
# Generate a bar plot showing the total population of races and poverty rate
merged_df.boxplot(by='Race', column=['Poverty Rate'], grid=False,figsize=(10, 15))

In [ ]:
# Generate a bar plot showing the total population of races and infections
merged_df.boxplot(by='Race', column=['Infections'], grid=False,figsize=(10, 15))